# Solutions to homework #4

In [1]:
import pandas as pd
import numpy as np
from tqdm.autonotebook import tqdm, trange
from sentence_transformers import SentenceTransformer

/var/folders/0g/qngbcycj71l5m10mjn_2bxxh0000gn/T/ipykernel_50150/96233274.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [2]:
github_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv'
data_url = f'{github_url}?raw=1'
df = pd.read_csv(data_url)
df = df.iloc[:300]

In [3]:
df.head()

,answer_llm,answer_orig,document,question,course
0,You can sign up for the course by visiting the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Where can I sign up for the course?,machine-learning-zoomcamp
1,You can sign up using the link provided in the...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Can you provide a link to sign up?,machine-learning-zoomcamp
2,"Yes, there is an FAQ for the Machine Learning ...",Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Is there an FAQ for this Machine Learning course?,machine-learning-zoomcamp
3,The context does not provide any specific info...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,Does this course have a GitHub repository for ...,machine-learning-zoomcamp
4,To structure your questions and answers for th...,Machine Learning Zoomcamp FAQ\nThe purpose of ...,0227b872,How can I structure my questions and answers f...,machine-learning-zoomcamp


In [4]:
model_name = 'multi-qa-mpnet-base-dot-v1'
embedding_model = SentenceTransformer(model_name)

/usr/local/Caskroom/miniconda/base/envs/llm-zoomcamp/lib/python3.11/site-packages/torch/backends/mps/__init__.py:24: UserWarning: Skipping device NVIDIA GeForce GT 750M that does not support Metal 2.0 (Triggered internally at /Users/runner/miniforge3/conda-bld/libtorch_1719361037267/work/aten/src/ATen/mps/MPSDevice.mm:97.)
  return torch._C._mps_is_available()
/usr/local/Caskroom/miniconda/base/envs/llm-zoomcamp/lib/python3.11/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [5]:
answer_llm = df.iloc[0].answer_llm

In [6]:
vector = embedding_model.encode(answer_llm)

## Q1. Getting the embeddings model: Solution to question #1

What's the first value of the resulting vector?

* **-0.42**
* -0.22
* -0.02
* 0.21

In [7]:
print(f'The solution to the first vector is {round(vector[0], 3):.2f}.')

The solution to the first vector is -0.42.


## Q2. Computing the dot product: Solution to question #2

Now for each answer pair, let's create embeddings and compute dot product between them. We will put the results (scores) into the `evaluations` list. What's the 75% percentile of the score?

* 21.67
* **31.67**
* 41.67
* 51.67

In [8]:
answers = df[['answer_llm', 'answer_orig']]

In [13]:
evaluations = []

for i in tqdm(range(answers.shape[0])):
    
    answer_orig = answers.iloc[i]['answer_orig']
    vector_orig = np.array(embedding_model.encode(answer_orig))
    
    answer_llm = answers.iloc[i]['answer_llm']
    vector_llm = np.array(embedding_model.encode(answer_llm))

    evaluations.append(vector_orig.dot(vector_llm))

  0%|          | 0/300 [00:00<?, ?it/s]

In [14]:
print(f'The 75% percentile of the score is: {round(np.percentile(np.array(evaluations), 75), 3)}.')

The 75% percentile of the score is: 31.674.


## Q3. Computing the cosine

What's the 75% cosine in the scores?

* 0.63
* 0.73
* **0.83**
* 0.93

In [15]:
def vector_norm(v):
    norm = np.sqrt((v * v).sum())
    return v / norm

In [16]:
evaluations_norm = []

for i in tqdm(range(answers.shape[0])):
    
    answer_orig = answers.iloc[i]['answer_orig']
    vector_orig = np.array(embedding_model.encode(answer_orig))
    vector_norm_orig = vector_norm(vector_orig)
    
    answer_llm = answers.iloc[i]['answer_llm']
    vector_llm = np.array(embedding_model.encode(answer_llm))
    vector_norm_llm = vector_norm(vector_llm)

    evaluations_norm.append(vector_norm_orig.dot(vector_norm_llm))

  0%|          | 0/300 [00:00<?, ?it/s]

In [17]:
print(f'The 75% percentile of the score is: {round(np.percentile(np.array(evaluations_norm), 75), 3)}.')

The 75% percentile of the score is: 0.836.
